# Tools Used in Pipeline

```mermaid
flowchart LR

%% =============================
%% PREPROCESSING
%% =============================
subgraph PRE["🛠️ Preprocessing Tools"]
    MP["⚙️ Mediapipe FaceMesh<br>(face alignment)"]
    CV2["⚙️ OpenCV<br>(resizing, cropping)"]
    PIL["🖼️ Pillow<br>(image I/O)"]
    TQDM["tqdm<br>(progress bars)"]
end

%% =============================
%% EMBEDDINGS
%% =============================
subgraph EMB["🧠 Embedding Extraction"]
    HF["🤗 HuggingFace Transformers<br>(CLIP ViT-H/14)"]
    NP["numpy<br>(matrix ops)"]
    PKL["pickle<br>(pretrained embeddings)"]
end

%% =============================
%% ML MODELS
%% =============================
subgraph ML["🧠 ML & Regression Models"]
    TORCH["torch<br>(MLP regressor)"]
    SKL["scikit-learn<br>(PCA, metrics)"]
end

%% =============================
%% DATA ENGINEERING
%% =============================
subgraph DATA["📊 Data Handling & Metadata"]
    PANDAS["pandas<br>(analytics, merging)"]
    JSON["json<br>(index files, metadata)"]
    PARQ["Parquet / Polars<br>(structured data)"]
end

%% =============================
%% VISUALIZATION
%% =============================
subgraph VIS["📈 Visualization"]
    MPL["matplotlib<br>(composite grids)"]
    ST["streamlit<br>(future app UI)"]
end

%% =============================
%% FUTURE / PLANNED
%% =============================
subgraph FUTURE["🚀 Planned Tools"]
    POLARS["📐 Polars<br>(fast dataframe ops)"]
    ONNX["🔧 ONNX Runtime<br>(optional accel)"]
    OPENCLIP["🧠 open_clip<br>(alt CLIP model)"]
    LGBM["🌲 LightGBM<br>(alt regressor)"]
    HF_SPACE["🤗 HF Spaces<br>(hosting)"]
end

%% =============================
%% CONNECTIONS (LANDSCAPE)
%% =============================

PRE --> EMB --> ML --> VIS
EMB --> DATA
DATA --> ML

PANDAS --> ML
NP --> TORCH
JSON --> DATA
PKL --> EMB

FUTURE --> ML
FUTURE --> DATA
FUTURE --> EMB
```

# 🛠️ Diagram 1 - FaceStats v4.0 High Level Pipeline

```mermaid
flowchart LR

A["📁 Raw Images<br>data/raw/<br><code>preprocess.py</code>"] 
    --> B["🖼️ Preprocessing<br>PIL Resize 512×512<br>No OpenCV Needed<br><code>preprocess.py</code>"]

B --> C["🧠 Embedding Extraction<br>HuggingFace CLIP / ViT<br>transformers + torch<br><code>embedding_extractor.py</code>"]

C --> D["👥 Attribute Models<br>Age / Gender / Ethnicity<br>HuggingFace pipelines<br><code>attribute_inference.py</code>"]

C --> E["💚 Attractiveness Regressor<br>PyTorch MLP<br><code>attractiveness_model.py</code>"]

D --> F["📊 Metadata Table Builder<br>Polars / Parquet<br><code>metadata_builder.py</code>"]

E --> F

F --> G["🧬 Composite Generator<br>PIL + NumPy Weighted Mean<br><code>composite_generator.py</code>"]

G --> H["📈 Dashboards<br>Streamlit + Matplotlib<br><code>app.py</code>"]


# 🖼️ Diagram 2 — Preprocessing Flow
*Shows exactly how images are normalized and resized.*

```mermaid
flowchart LR

A["📁 Input Images<br>data/raw/"] 
    --> B["🔍 File Loader<br>PIL Image.open<br><code>preprocess.py</code>"]

B --> C["🧽 Clean / Normalize<br>Convert to RGB<br><code>PIL</code>"]

C --> D["📏 Resize 512×512<br>High-quality bicubic<br><code>PIL.Image.resize</code>"]

D --> E["💾 Save JPEG<br>data/processed/preproc/<br><code>preprocess.py</code>"]


# 🧠 Diagram 3 — Embedding Extraction
*Uses HuggingFace, zero OpenCV/ONNX.*

```mermaid
flowchart LR

A["🖼️ Preprocessed Images<br>data/processed/preproc/"] 
    --> B["📥 Tokenizer<br>CLIPProcessor<br><code>transformers</code>"]

B --> C["🧠 Forward Pass<br>CLIPModel or ViTModel<br>torch + transformers<br><code>embedding_extractor.py</code>"]

C --> D["📏 L2 Normalize<br>512-D vector<br><code>NumPy / Torch</code>"]

D --> E["💾 Store Embeddings<br>embeddings.parquet<br><code>polars</code>"]

# 👥 Diagram 4 — Attribute Models
*No ONNX — pure HF pipelines.*

```mermaid
flowchart LR

A["🧠 Embeddings + Images"] 
    --> B["👤 HF Age Model<br>AutoModelForImageClassification<br><code>attribute_inference.py</code>"]

A --> C["👥 HF Gender Model<br>transformers pipeline"]

A --> D["🌍 HF Ethnicity Model<br>image-classification model"]

B --> E["📄 Attributes Table<br>attributes.parquet"]
C --> E
D --> E


# 💚 Diagram 5 — Attractiveness Model
*Your in-house regression model.*

```mermaid
flowchart LR

A["🔢 Embeddings (512-D)"] 
    --> B["🧠 PyTorch MLP Regressor<br>torch.nn.Sequential<br><code>attractiveness_model.py</code>"]

B --> C["📈 Score Output<br>scores.parquet"]

C --> D["📦 Merge into Metadata<br><code>metadata_builder.py</code>"]

# 📊 Diagram 6 — Metadata Table Builder
*Combines embeddings, attributes, scores into one dataset.*

```mermaid
flowchart LR

A["🔢 Embeddings.parquet"] 
    --> E["📊 Merge Tables<br>Polars DataFrame<br><code>metadata_builder.py</code>"]

B["👤 Attributes.parquet"] --> E
C["💚 Scores.parquet"] --> E

E --> F["💾 Master Table<br>master.parquet"]

# 🧬 Diagram 7 — Composite Image Pipeline
*Produces PCA or weighted-mean faces.*

```mermaid
flowchart LR

A["📊 Master Metadata<br>master.parquet"] 
    --> B["📋 Filter Selection<br>age, gender, attractiveness, ethnicity<br><code>composite_generator.py</code>"]

B --> C["📥 Load Images<br>PIL image loader"]

C --> D["➕ NumPy Stack<br>Convert to arrays<br><code>numpy</code>"]

D --> E["📊 Mean / Weighted Mean<br>PCA optional<br><code>NumPy / sklearn PCA</code>"]

E --> F["🖼️ Save Composite<br>data/processed/composites/<br><code>composite_generator.py</code>"]

# 📦 Idealized Repository Structure

```text
FaceStats/
├── LICENSE
├── README.md
├── requirements.txt
├── schematics.ipynb
├── x.ipynb

├── config/
│   └── ...                               # Configuration files

├── data/
│   ├── attributes/
│   ├── embeddings/
│   ├── interim/
│   │   └── checkpoints/
│   ├── preprocessed/
│   ├── processed/
│   │   ├── metadata/
│   │   ├── attributes_clean.parquet
│   │   ├── attributes_flags.parquet
│   │   ├── attributes_with_clusters.parquet
│   │   ├── attributes_with_meta.parquet
│   │   ├── attributes.parquet
│   │   └── embeddings_clip.parquet
│   ├── raw/
│   │   ├── a small sample (700 images)/
│   │   ├── SFHQ_pt4_00000xxx.jpg…
│   └── # TODO: data/raw/part1, part2, part3, part4 (full dataset)

├── models_insightface/
│   └── ...                               # InsightFace model weights, configs

├── notebooks/
│   ├── 01_preprocess.ipynb
│   ├── 02_embeddings.ipynb
│   ├── 03_attributes.ipynb
│   ├── 03_labels.ipynb
│   ├── 04_visualize_attributes.ipynb
│   ├── 05_ethnicity_clusters.ipynb
│   └── # TODO: fs07_age_gender_ethnicity_inference.ipynb
│   └── # TODO: fs08_fairface_alignment.ipynb

├── src/
│   ├── attributes/
│   │   ├── age_infer.py
│   │   ├── face_attributes.py
│   │   └── # TODO: ethnicity_infer.py
│   │   └── # TODO: gender_infer.py

│   ├── composite/
│   │   ├── composite_generator.py
│   │   └── # TODO: composite_filters.py
│   │   └── # TODO: composite_explorer.py

│   ├── data_utils/
│   │   ├── constants.py
│   │   ├── filters.py
│   │   ├── io_utils.py
│   │   └── # TODO: dataset_splits.py
│   │   └── # TODO: data_validation.py

│   ├── embeddings/
│   │   ├── embed_clip.py
│   │   └── # TODO: embed_openclip.py   # Only if needed

│   ├── learning/
│   │   ├── learning_curves.py
│   │   └── # TODO: trainers.py
│   │   └── # TODO: loss_functions.py

│   ├── metadata/
│   │   └── build_master.py

│   ├── models/
│   │   ├── attractiveness_model.py
│   │   ├── train_attractiveness.py
│   │   └── # TODO: multi_attribute_model.py
│   │   └── # TODO: fairness_model.py

│   ├── pipeline/
│   │   ├── __init__.py
│   │   ├── preprocess.py
│   │   └── # TODO: full_pipeline_runner.py

│   ├── visualization/
│   │   ├── app.py
│   │   ├── __init__.py
│   │   └── # TODO: dashboards/
│   │       └── # TODO: attractiveness_dashboard.py
│   │       └── # TODO: embeddings_explorer.py
│   │       └── # TODO: composite_gallery.py

│   └── # TODO: utils/
│       └── # TODO: logger.py
│       └── # TODO: config_loader.py
```
